In [1]:
from fastai.text.all import *
from fastai.callback.fp16 import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('mtsamples.csv')

In [3]:
df.head(3)

,id,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it...","allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,"
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climbing stairs, difficulty with airline seats, tying shoes, used to public seating, and lifting objects off the floor. He exercises three times a week at home and does cardio. He has difficulty walking two blocks or five flights of stairs. Difficulty with snoring. He has muscle and joint pains including knee pain, back pain, foot and ankle pain, and swelling. He has gastroesophageal reflux disease.,PAST SURGICAL HISTORY:, Includes reconstructive surgery on his right hand 13 years ago. ,SOCIAL HISTORY:, He is currently single. He has about te...","bariatrics, laparoscopic gastric bypass, weight loss programs, gastric bypass, atkin's diet, weight watcher's, body weight, laparoscopic gastric, weight loss, pounds, months, weight, laparoscopic, band, loss, diets, overweight, lost"
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC today. He is a very pleasant gentleman who is 42 years old, 344 pounds. He is 5'9"". He has a BMI of 51. He has been overweight for ten years since the age of 33, at his highest he was 358 pounds, at his lowest 260. He is pursuing surgical attempts of weight loss to feel good, get healthy, and begin to exercise again. He wants to be able to exercise and play volleyball. Physically, he is sluggish. He gets tired quickly. He does not go out often. When he loses weight he always regains it and he gains back more than he lost. His biggest...","bariatrics, laparoscopic gastric bypass, heart attacks, body weight, pulmonary embolism, potential complications, sleep study, weight loss, gastric bypass, anastomosis, loss, sleep, laparoscopic, gastric, bypass, heart, pounds, weight,"


In [4]:
df = df[df['medical_specialty'] != ' Surgery']

In [5]:
df['medical_specialty'].unique()

array([' Allergy / Immunology', ' Bariatrics',
       ' Cardiovascular / Pulmonary', ' Neurology', ' Dentistry',
       ' Urology', ' General Medicine', ' Speech - Language',
       ' SOAP / Chart / Progress Notes', ' Sleep Medicine',
       ' Rheumatology', ' Radiology', ' Psychiatry / Psychology',
       ' Podiatry', ' Physical Medicine - Rehab',
       ' Pediatrics - Neonatal', ' Pain Management', ' Orthopedic',
       ' Ophthalmology', ' Office Notes', ' Obstetrics / Gynecology',
       ' Neurosurgery', ' Nephrology', ' Letters',
       ' Lab Medicine - Pathology', ' IME-QME-Work Comp etc.',
       ' Hospice - Palliative Care', ' Hematology - Oncology',
       ' Gastroenterology', ' ENT - Otolaryngology', ' Endocrinology',
       ' Emergency Room Reports', ' Discharge Summary',
       ' Diets and Nutritions', ' Dermatology',
       ' Cosmetic / Plastic Surgery', ' Consult - History and Phy.',
       ' Chiropractic', ' Autopsy'], dtype=object)

In [6]:
ms_data = df.groupby(['medical_specialty'])

In [7]:
for cat, data in ms_data:
    print(cat, len(data))

 Allergy / Immunology 7
 Autopsy 8
 Bariatrics 18
 Cardiovascular / Pulmonary 372
 Chiropractic 14
 Consult - History and Phy. 516
 Cosmetic / Plastic Surgery 27
 Dentistry 27
 Dermatology 29
 Diets and Nutritions 10
 Discharge Summary 108
 ENT - Otolaryngology 98
 Emergency Room Reports 75
 Endocrinology 19
 Gastroenterology 230
 General Medicine 259
 Hematology - Oncology 90
 Hospice - Palliative Care 6
 IME-QME-Work Comp etc. 16
 Lab Medicine - Pathology 8
 Letters 23
 Nephrology 81
 Neurology 223
 Neurosurgery 94
 Obstetrics / Gynecology 160
 Office Notes 51
 Ophthalmology 83
 Orthopedic 355
 Pain Management 62
 Pediatrics - Neonatal 70
 Physical Medicine - Rehab 21
 Podiatry 47
 Psychiatry / Psychology 53
 Radiology 273
 Rheumatology 10
 SOAP / Chart / Progress Notes 166
 Sleep Medicine 20
 Speech - Language 9
 Urology 158


In [8]:
filt_data = ms_data.filter(lambda x: x.shape[0] > 50)
new_filt_data = filt_data.groupby(['medical_specialty']).head(100).reset_index(drop=True)
final_data = new_filt_data.groupby(['medical_specialty'])

In [9]:
for cat, data in final_data:
    print(cat, len(data))

 Cardiovascular / Pulmonary 100
 Consult - History and Phy. 100
 Discharge Summary 100
 ENT - Otolaryngology 98
 Emergency Room Reports 75
 Gastroenterology 100
 General Medicine 100
 Hematology - Oncology 90
 Nephrology 81
 Neurology 100
 Neurosurgery 94
 Obstetrics / Gynecology 100
 Office Notes 51
 Ophthalmology 83
 Orthopedic 100
 Pain Management 62
 Pediatrics - Neonatal 70
 Psychiatry / Psychology 53
 Radiology 100
 SOAP / Chart / Progress Notes 100
 Urology 100


In [10]:
data_df = new_filt_data[['transcription', 'medical_specialty']]

In [11]:
dls_lm = DataBlock(
                blocks = TextBlock.from_df('transcription', is_lm=True),
                get_x=ColReader('text'), 
                splitter=RandomSplitter(0.1)
).dataloaders(data_df, bs=64)    
dls_lm.show_batch(max_n=3)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


,text,text_
0,"xxbos xxup preoperative xxup diagnosis : , xxmaj right renal mass . ,postop xxup diagnosis : , xxmaj right renal mass . ,procedure xxup performed : , xxmaj laparoscopic right radical nephrectomy . ,estimated xxup blood xxup loss : , 100 ml . ,x - rays : , none . ,specimens : , xxmaj right radical nephrectomy specimen . ,complications : , none . ,anesthesia : , general endotracheal . ,drains :","xxup preoperative xxup diagnosis : , xxmaj right renal mass . ,postop xxup diagnosis : , xxmaj right renal mass . ,procedure xxup performed : , xxmaj laparoscopic right radical nephrectomy . ,estimated xxup blood xxup loss : , 100 ml . ,x - rays : , none . ,specimens : , xxmaj right radical nephrectomy specimen . ,complications : , none . ,anesthesia : , general endotracheal . ,drains : ,"
1,foraminal disc osteophyte complex with right - sided uncovertebral and apophyseal joint arthrosis . xxmaj the disc osteophyte complex measures approximately 5 mm in its xxup ap dimension . xxmaj there is minimal posterior annular bulging measuring approximately 2 mm . xxmaj the xxup ap diameter of the central canal has been narrowed to 9 mm . xxup csf remains present surrounding the cord . xxmaj there is probable radicular impingement upon,disc osteophyte complex with right - sided uncovertebral and apophyseal joint arthrosis . xxmaj the disc osteophyte complex measures approximately 5 mm in its xxup ap dimension . xxmaj there is minimal posterior annular bulging measuring approximately 2 mm . xxmaj the xxup ap diameter of the central canal has been narrowed to 9 mm . xxup csf remains present surrounding the cord . xxmaj there is probable radicular impingement upon the
2,": xxunk xxrep 3 i : xxunk xxup rem : xxunk patient had relatively good sleep xxunk , except for excessive waking . ,respiratory xxup measurements : , xxmaj total apnea / hypopnea 75 , age index 12.3 per hour . xxup rem age index 15 per hour . xxmaj total arousal 101 , arousal index 15.6 per hour . xxmaj oxygen desaturation was down to 88 % . xxmaj longest event","xxunk xxrep 3 i : xxunk xxup rem : xxunk patient had relatively good sleep xxunk , except for excessive waking . ,respiratory xxup measurements : , xxmaj total apnea / hypopnea 75 , age index 12.3 per hour . xxup rem age index 15 per hour . xxmaj total arousal 101 , arousal index 15.6 per hour . xxmaj oxygen desaturation was down to 88 % . xxmaj longest event 35"


In [12]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])

In [13]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.431013,4.256220,0.329499,70.542854,00:30


In [14]:
learn.save('mtsamples_1epoch')

Path('models/mtsamples_1epoch.pth')

In [15]:
learn = learn.load('mtsamples_1epoch')

In [16]:
learn.unfreeze()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.490884,3.323530,0.416983,27.758163,00:30


In [17]:
learn.save_encoder('mtsamples_finetuned')

In [18]:
dls_clas = DataBlock(
                blocks = (TextBlock.from_df('transcription'), CategoryBlock),
                get_x=ColReader('text'),
                get_y=ColReader('medical_specialty'),
                splitter=RandomSplitter(0.2)
).dataloaders(data_df, bs=64)
dls_clas.show_batch(max_n=3)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck
Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


,text,category
0,"xxbos xxup type xxup of xxup consultation : , xxmaj wound care consult . ,history xxup of xxup present xxup illness : , xxmaj the patient is a 62-year - old woman with a past medical history significant for prior ileojejunal bypass for weight loss ( 1980 ) and then subsequent gastric banding ( 2002 xxmaj dr . xxup x ) who was transferred to this facility following a complicated surgical and postoperative course after takedown of the prior xxunk and bypass ( 07 / 08 / 2008 , xxmaj dr . xxup y ) . xxmaj the patient has been followed by xxmaj cardiothoracic xxmaj surgery ( dr . xxup z ) as an outpatient . xxmaj she had a history of daily postprandial vomiting , regurgitation , and heartburn . xxmaj she underwent a preop assessment of her xxup gerd and postprandial vomiting including nuclear gastric emptying studies",Consult - History and Phy.
1,"xxbos xxup identification xxup of xxup patient : , xxmaj this is a 31-year - old female who was referred by herself . xxmaj she was xxunk seen at xxmaj counseling xxmaj center . xxmaj she is a reliable historian . ,chief xxup complaint : , "" i 'm bipolar and i have severe anxiety disorder . i have posttraumatic stress syndrome . "" , history xxup of xxup present xxup illness : , xxmaj at age 19 , xxmaj ms . xxmaj abc had a recurrence of memories . xxmaj her father had molested her , and the memories returned . xxmaj in 1992 , at the age of 18 , she entered her first abusive marriage . xxmaj she was xxunk and her husband shared her sexually with his friends . xxmaj this lasted until age 24 . xxmaj the second marriage was age 26 , her second",Psychiatry / Psychology
2,"xxbos xxup reason xxup for xxup referral : , xxmaj the patient is a 58-year - old african - american right - handed female with 16 years of education who was referred for a neuropsychological evaluation by xxmaj dr . xxup x. xxmaj she is presenting for a second opinion following a recent neuropsychological evaluation that was ordered by her former place of employment that suggested that she was in the "" early stages of a likely dementia "" and was thereafter terminated from her position as a psychiatric nurse . a comprehensive evaluation was requested to assess current cognitive functioning and assist with diagnostic decisions and treatment planning . xxmaj note that this evaluation was undertaken as a clinical exam and intended for the purposes of xxunk with treatment planning . xxmaj the patient was fully informed about the nature of this evaluation and intended use of the results",Psychiatry / Psychology


In [19]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy)

In [20]:
learn = learn.load_encoder('mtsamples_finetuned')

In [21]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,3.104645,2.932493,0.097035,00:08
1,2.803523,2.463584,0.312668,00:08
2,2.523131,2.165913,0.369272,00:07
3,2.287578,2.029305,0.428571,00:07
4,2.099773,1.924459,0.442049,00:08
5,1.962154,1.878082,0.458221,00:08
6,1.845803,1.819276,0.474394,00:08
7,1.742889,1.804273,0.466307,00:08
8,1.685616,1.799267,0.477089,00:08
9,1.636451,1.807369,0.474394,00:08
